In [42]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import pandas

seed = 7
np.random.seed(seed)
def string2bits(s=''):
    return [bin(ord(x))[2:].zfill(8) for x in s]
def bits2string(b=None):
    return ''.join([chr(int(x, 2)) for x in b])
dataframe = pandas.read_csv("hash_md5_dataset_sample.csv", header=None)
dataset = dataframe.values
x = dataset[:, 0:2]
X = []
Y= []
x_test=[]
y_test=[]

for i in range(4000):
    X.append(((''.join(string2bits(x[i][0])))))

for i in range(4000):
    Y.append(((''.join(string2bits(x[i][1])))))

for i in range(4000,5000):
    x_test.append(((''.join(string2bits(x[i][0])))))

for i in range(4000,5000):
    y_test.append(((''.join(string2bits(x[i][1])))))


x_train = np.array([[int(x) for x in y] for y in X])
y_train = np.array([[int(x) for x in y] for y in Y])

x_test = np.array([[int(x) for x in y] for y in x_test])
y_test = np.array([[int(x) for x in y] for y in y_test])

model = Sequential()
model.add(Dense(256, input_dim=256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(160, activation='softmax'))




model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
classes = model.predict(x_test, batch_size=32)
print(score)

Epoch 1/20
4000/4000 [==============================] - 0s - loss: 2.3353 - acc: 0.5298     
Epoch 2/20
4000/4000 [==============================] - 0s - loss: 2.3208 - acc: 0.5298     
Epoch 3/20
4000/4000 [==============================] - 0s - loss: 2.3170 - acc: 0.5298     
Epoch 4/20
4000/4000 [==============================] - 0s - loss: 2.3145 - acc: 0.5298     
Epoch 5/20
4000/4000 [==============================] - 0s - loss: 2.3128 - acc: 0.5298     
Epoch 6/20
4000/4000 [==============================] - 0s - loss: 2.3112 - acc: 0.5298     
Epoch 7/20
4000/4000 [==============================] - 0s - loss: 2.3097 - acc: 0.5298     
Epoch 8/20
4000/4000 [==============================] - 0s - loss: 2.3082 - acc: 0.5298     
Epoch 9/20
4000/4000 [==============================] - 0s - loss: 2.3067 - acc: 0.5298     
Epoch 10/20
4000/4000 [==============================] - 0s - loss: 2.3052 - acc: 0.5298     
Epoch 11/20
4000/4000 [==============================] - 0s - loss: 2

In [43]:
print(classes[1])


[  2.48663468e-07   1.27990562e-02   5.97839756e-03   4.45723720e-03
   5.18938620e-03   6.48913486e-03   8.51844810e-03   7.69373728e-03
   2.20308252e-07   9.77923162e-03   7.51849543e-03   8.36206786e-03
   5.56680700e-03   4.96768253e-03   5.30790212e-03   5.70505578e-03
   2.19651596e-07   9.84007958e-03   7.64309941e-03   5.68766287e-03
   6.12042565e-03   5.57255838e-03   6.53448608e-03   7.33633572e-03
   2.19585829e-07   1.04742376e-02   8.14778171e-03   6.90847822e-03
   5.76620083e-03   7.32373958e-03   5.46052353e-03   6.00481220e-03
   2.26239507e-07   1.07088061e-02   6.87126676e-03   5.53815300e-03
   5.52689284e-03   7.93199148e-03   5.56267425e-03   7.61373620e-03
   2.31924460e-07   9.60363075e-03   8.06570053e-03   7.74909602e-03
   4.48395777e-03   5.31754177e-03   4.74938378e-03   7.45569030e-03
   2.14814790e-07   9.31403879e-03   8.49321485e-03   7.15703983e-03
   4.43278672e-03   6.36777841e-03   7.99301453e-03   8.02673679e-03
   2.16110763e-07   1.02237370e-02

In [33]:
x_test

array([[0, 1, 1, ..., 1, 1, 0],
       [0, 1, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 1],
       ..., 
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 1],
       [0, 0, 1, ..., 1, 1, 0]])